## Lab 8 - Logic Gate Network

Zaimplementuj Logic Gate Network, czyli sieć warstwową, w której każdy "neuron" wykorzystuje jeden z operatorów logicznych na wybranych wejściach. W trakcie uczenia, możliwa jest modyfikacja wyboru wejść i zmiana operatora. Wygeneruj zbiór danych na zainicjalizowanym modelu, a następnie spróbuj nauczyć inny model, wykorzystując np. algorytm genetyczny.

### Logic Gate

In [73]:
class LogicGate():
    def __init__(self, operator, input_elements):
        self.input_elements = input_elements
        self.operator_name = operator
        self.operator_dict = {'AND': self._and_operator, 'OR': self._or_operator, 'XOR': self._xor_operator, 'NAND': self._nand_operator, 'NOR': self._nor_operator}
        self.operator = self.operator_dict[operator]
        
    def _and_operator(self, inp1, inp2):
        if inp1 == 1 and inp2 ==1:
            return 1
        else:
            return 0
            
    def _or_operator(self, inp1, inp2):
        if any([inp1, inp2]):
            return 1
        else:
            return 0
            
    def _xor_operator(self, inp1, inp2):
        if inp1 != inp2:
            return 1
        else:
            return 0
        
    def _nand_operator(self, inp1, inp2):
        if inp1 == inp2 and inp1 == 1:
            return 0
        else:
            return 1
        
    def _nor_operator(self, inp1, inp2):
        if inp1 == inp2 and inp1 == 0:
            return 1
        else:
            return 0
        
    def calculate(self, inp1, inp2):
        return self.operator(inp1, inp2)
        
    def change_operator(self, new_operator):
        self.operator = self.operator_dict[new_operator]
            
    def change_inputs(self, new_inputs):
        self.input_elements = new_inputs
            
    def return_elements(self):
        return self.input_elements
    
    def __repr__(self):
        return f'operator : {self.operator_name}, inputs: {self.input_elements}'

### Logic Gate Network

In [74]:
import numpy as np

class LogicGateNetwork():
    def __init__(self, input_size):
        self.layers = []
        self.layer_dict = {}
        self.size_of_network = {0: input_size}
        self.input_size = input_size
        
    def add_inputs(self, input_list):
        if len(input_list) != self.input_size:
            raise Exception("Length of input list does not equal input size for this network")
        self.layer_dict[0] = input_list
    
    def add_layer(self, number, gate_list):
        self.layer_dict[number] = gate_list
        self.size_of_network[number] = len(gate_list)      
    
    def calculate(self):
        current_layer = 1
        list_of_results = self.layer_dict[0]
        for current_layer in range(1, len(self.layer_dict)):
            operators = self.layer_dict[current_layer]
            list_of_current_results = []
            for operator in operators:
                inp_ind1, inp_ind2 = operator.return_elements()
                inp1, inp2 = list_of_results[inp_ind1], list_of_results[inp_ind2]
                result = operator.calculate(inp1, inp2)
                list_of_current_results.append(result)
            list_of_results = list_of_current_results
            current_layer = current_layer + 1
        return list_of_results
    
    def __repr__(self):
        string = ''
        for layer_num, layer_el in self.layer_dict.items():
            string = string + f'layer {layer_num}: {str(layer_el)} \n'
        return string

### Genetic Algorithm for Logic Gate Network

In [210]:
import random
from copy import deepcopy

class GAForLogicGateNetwork():
    
    def __init__(self, example_network, p_mutation=0.4, n_population=500, n_iter=1000):
        self.size_dict = example_network.size_of_network
        self.input_size = example_network.input_size
        self.p_mutation = p_mutation
        self.n_population = n_population
        self.n_iter = n_iter
        
        self.population = [self.generate_individual() for _ in range(n_population)]
    
    def generate_individual(self):
        individual = LogicGateNetwork(self.input_size)
        for layer_num, size_of_layer in self.size_dict.items():
            if layer_num==0:
                continue
            layer = []
            for _ in range(size_of_layer):
                random_operator = np.random.choice(['AND', 'OR', 'XOR', 'NAND', 'NOR'])
                random_inputs = np.random.choice(range(self.size_dict[layer_num-1]), 2, False)
                layer.append(LogicGate(random_operator, tuple(random_inputs)))
            individual.add_layer(layer_num, layer)
        return individual
    
    def crossover(self, individual1, individual2):
        crossover_layer = np.random.randint(1, len(self.size_dict)-1)
        for i in range(1, len(self.size_dict)):
            if i<=crossover_layer:
                individual1.layer_dict[i], individual2.layer_dict[i] = individual2.layer_dict[i], individual1.layer_dict[i]
        return individual1, individual2
                
    def fitness_function(self, outputs, values):
        return sum(np.array(outputs) != np.array(values))
    
    def mutation(self, individual):
        # wybieramy warstwę
        random_layer = np.random.randint(1, len(self.size_dict))
        # wybieramy komórkę do zmiany
        random_gate = np.random.randint(0, self.size_dict[random_layer])
        random_operator = np.random.choice(['AND', 'OR', 'XOR', 'NAND', 'NOR'])
        random_inputs = np.random.choice(range(self.size_dict[random_layer-1]), 2, False)
        individual.layer_dict[random_layer][random_gate] = LogicGate(random_operator, tuple(random_inputs))

        return individual
    
    def evaluate_individual(self, individual, xs, ys):
        result = []
        for x in xs:
            individual.add_inputs(x)
            result.append(*individual.calculate()) 
        return self.fitness_function(result, ys)
            
    def optimize(self, x, y):
        elitism_percent = 0.1
        n_elite = n_elite = max(1, int(self.n_population * elitism_percent))  # Ensure at least 1 elite individual
        sorted_population = sorted(self.population, key=lambda ind: self.evaluate_individual(ind, x, y))
        best_individual_so_far = [deepcopy(sorted_population[0]), self.evaluate_individual(sorted_population[0], x, y)]
        print('Initial Best Fitness:', self.evaluate_individual(sorted_population[0], x, y))
        
        for _ in range(self.n_iter):
            elite_individuals = deepcopy(sorted_population[:n_elite])
            
            new_population = []
            for _ in range((self.n_population-n_elite)//2):
                parent1, parent2 = random.sample(sorted_population, 2)
                
                if random.random() < self.p_mutation:
                    offspring1 = self.mutation(parent1)
                    offspring2 = self.mutation(parent2)
                else:
                    offspring1, offspring2 = self.crossover(parent1, parent2)
                    
                new_population.append(offspring1)
                new_population.append(offspring2)
            
            combined_population = elite_individuals + new_population
            
            sorted_population = sorted(combined_population, key=lambda ind: self.evaluate_individual(ind, x, y))[:self.n_population]
            best_individual = deepcopy(sorted_population[0])
            best_score = self.evaluate_individual(best_individual, x, y)
            
            if best_score > best_individual_so_far[1]:
                best_individual_so_far = [best_individual, best_score]
            
            if best_score == 0:
                break
        
        sorted_population = sorted(sorted_population, key=lambda ind: self.evaluate_individual(ind, x, y))[:self.n_population]
        best_individual = sorted_population[0]
        best_score = self.evaluate_individual(best_individual, x, y)
        
        print("Final Best Fitness:", best_score)
        
        return best_individual, best_score

### Application

In [211]:
# Network Architecture

first_layer = [LogicGate('AND', (0,2)), LogicGate('NOR', (1,4)), LogicGate('XOR', (3,5))]
second_layer = [LogicGate('NAND', (0,1)), LogicGate('OR', (0,1)), LogicGate('OR', (1,2)), LogicGate('AND', (1,2))]
third_layer = [LogicGate('AND', (0,1)), LogicGate('AND', (1,2)), LogicGate('NOR', (2,3))]
forth_layer = [LogicGate('AND', (0,1)), LogicGate('OR', (1,2))]
fifth_layer = [LogicGate('NOR', (0,1))]

inputs = [1,1,0,0,1,0]

siec = LogicGateNetwork(6)
siec.add_layer(1, first_layer)
siec.add_layer(2, second_layer)
siec.add_layer(3, third_layer)
siec.add_layer(4, forth_layer)
siec.add_layer(5, fifth_layer)

siec.add_inputs(inputs)
siec.calculate()

[0]

In [212]:
# Dataset creation

from itertools import product

x = list(product([0,1], repeat=6))
y = []
for el_x in x:
    siec.add_inputs(el_x)
    y.append(*siec.calculate())
print(y)

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [217]:
# Optimization with genetic algorithm

optimizer = GAForLogicGateNetwork(siec, 0.5, 1000, 1000)
found_network, score  = optimizer.optimize(x, y)

Initial Best Fitness: 14
Final Best Fitness: 0


In [218]:
found_network

layer 1: [operator : XOR, inputs: (5, 3), operator : AND, inputs: (0, 2), operator : OR, inputs: (1, 4)] 
layer 2: [operator : XOR, inputs: (0, 1), operator : OR, inputs: (1, 2), operator : NAND, inputs: (2, 1), operator : NAND, inputs: (0, 2)] 
layer 3: [operator : AND, inputs: (2, 0), operator : AND, inputs: (1, 0), operator : XOR, inputs: (3, 2)] 
layer 4: [operator : OR, inputs: (2, 0), operator : XOR, inputs: (0, 2)] 
layer 5: [operator : XOR, inputs: (0, 1)] 
layer 0: (1, 1, 1, 1, 1, 1) 

In [219]:
found_y = []
for el_x in x:
    found_network.add_inputs(el_x)
    found_y.append(*found_network.calculate())
print(found_y)
print(y)

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Architektura sieci znaleziona przez algorytm genetyczny różni się od tej podanej, ale daje ona takie wyniki na zbiorze danych.